In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("Vietnam_Movies.xlsx")

In [3]:
df['Year_of_Release'] = df['Release Date']
df['Year_of_Release'] = pd.to_datetime(df['Year_of_Release'])
df['Year_of_Release'] = df['Year_of_Release'].dt.year
df['Year_of_Release'] = df['Year_of_Release'].fillna(0)
df['Year_of_Release'] = df['Year_of_Release'].astype(int)

In [4]:
revenue_by_year_and_company = pd.DataFrame(columns=['release_year', 'company', 'revenue'])

revenue_by_company_year = {}

for index, row in df.iterrows():
    if isinstance(row['Production Companies'], str):
        companies = row['Production Companies']
        year = row['Year_of_Release']
        revenue = row['Revenue']
        
        cleaned = companies.replace("[", "").replace("]", "").replace("'", "").replace('"', "")
        company_list = cleaned.split(",")
        company_list = [company.strip() for company in company_list]
        
        for company in company_list:
            if company not in revenue_by_company_year:
                    revenue_by_company_year[company] = {}
                    
            if year not in revenue_by_company_year[company]:
                revenue_by_company_year[company][year] = revenue

revenue_data = []
for company, year_revenue in revenue_by_company_year.items():
    for year, revenue in year_revenue.items():
        revenue_data.append([company, year, revenue])      

revenue_by_year_and_company = pd.DataFrame(revenue_data, columns=['company', 'release_year', 'revenue'])


In [5]:
revenue_by_year_and_company_sorted = revenue_by_year_and_company.groupby('company', as_index=False).apply(lambda x: x.sort_values('release_year'))

revenue_data = [] 
        
for company, group in revenue_by_year_and_company_sorted.groupby('company'):
    if company != '':
        pre_revenue = 0
        pre_year = 1989
        
        for index, row in group.iterrows():
            year = row['release_year']
            revenue = row['revenue']
            
            for y in range(pre_year + 1, year):
                if (y >= 1990) and (y <= 2024):
                    revenue_data.append([company, y, pre_revenue])
            
            revenue_data.append([company, year, revenue + pre_revenue])
            pre_revenue = revenue + pre_revenue
            pre_year = year
            
        for y in range(pre_year + 1, 2025):
            if (y >= 1990) and (y <= 2024):
                revenue_data.append([company, y, pre_revenue])
        
revenue_by_year_and_company = pd.DataFrame(revenue_data, columns=['company', 'year', 'revenue'])

C:\Users\HP\AppData\Local\Temp\ipykernel_11636\811819437.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  revenue_by_year_and_company_sorted = revenue_by_year_and_company.groupby('company', as_index=False).apply(lambda x: x.sort_values('release_year'))


In [6]:
revenue_by_year_and_company.to_excel('summary3.xlsx', index=False)